In [1]:
import pandas as pd

In [2]:
## Open file
bracken_file=pd.read_csv(filepath_or_buffer='/Users/jjpc/PiconCossio/UNI-SEQ/16S/G1_1/G1_1_1_G1_1_2_G.bracken_output' ,delimiter='\t')
bracken_file.head()

,name,taxonomy_id,taxonomy_lvl,kraken_assigned_reads,added_reads,new_est_reads,fraction_total_reads
0,uncultured,45380,G,20757,771,21528,0.06855
1,Eubacterium,45378,G,604,228,832,0.00265
2,Pseudoramibacter,45379,G,100,25,125,0.00040
3,Acetobacterium,45377,G,65,11,76,0.00024
4,Anaerofustis,45375,G,1085,55,1140,0.00363


In [7]:
bracken_file[bracken_file["fraction_total_reads"] >= 0.1]

,name,taxonomy_id,taxonomy_lvl,kraken_assigned_reads,added_reads,new_est_reads,fraction_total_reads
